In [39]:
import matplotlib as mpl
from matplotlib.widgets import RectangleSelector

mpl.use('Qt5Agg')
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
import cv2
import open3d
from intrinsics import Intrinsics
from tqdm.notebook import tqdm
from utils import pcd_to_depth_map, pcd_to_volume
import math
import scipy
from scipy.spatial.transform import Rotation

In [40]:
fps = 30
test_name = "test1_depth_normal"
scale = 0.00025
paths = glob.glob(os.path.join("koen_mri_room", test_name, "*.png"))
filename = paths[0].split("\\")[-1]

depth = np.asanyarray(open3d.io.read_image(paths[0])) * scale
depth = open3d.geometry.Image(depth.astype(np.float32))

# rgb = cv2.imread(os.path.join("koen_mri_room", test_name, "rgb", filename))
rgb = np.zeros(np.asarray(depth).shape)

In [41]:
paths[0]

'koen_mri_room\\test1_depth_normal\\frame_0_1710933553.519603.png'

In [42]:
figs, axs = plt.subplots(1, 2, figsize=(10, 5))
# axs[0].imshow(cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB))
axs[0].imshow(rgb)
axs[0].axis('off')
axs[1].imshow(depth, cmap="gray")
axs[1].axis('off')
plt.show()

In [5]:
# points = []
# 
# def mouse_event(event):
#     global points
#     global rgb_copy
# 
#     points.append([int(event.xdata), int(event.ydata)])
# 
#     if len(points) == 3:
#         plt.close()
# 
# fig = plt.figure(figsize=(10, 10))
# cid = fig.canvas.mpl_connect('button_press_event', mouse_event)
# # plt.imshow(cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB))
# plt.imshow(depth)
# plt.show()
# 
# # rgb_copy = rgb.copy()
# depth_copy = np.asarray(depth).copy()
# for p in points:
#     # marked_image = cv2.circle(rgb_copy, p, 3, (0, 255, 0), 2)
#     marked_image = cv2.circle(depth_copy, p, 3, (0, 255, 0), 2)
# 
# fig = plt.figure(figsize=(10, 10))
# # plt.imshow(cv2.cvtColor(rgb_copy, cv2.COLOR_BGR2RGB))
# plt.imshow(marked_image)
# plt.show()

In [43]:
points = [[332, 745], [586, 733], [528, 520]]

In [44]:
fig, ax = plt.subplots()
ax.imshow(depth, cmap="gray")
roi = None

def line_select_callback(eclick, erelease):
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    global roi
    roi = [[int(x1), int(y1)], [int(x2), int(y2)]]
    plt.close()

rs = RectangleSelector(ax, line_select_callback,
                       useblit=False, button=[1],
                       minspanx=5, minspany=5, spancoords='pixels',
                       interactive=True)


plt.connect("key_press_event", rs)
plt.show()

In [45]:
# roi = [[424, 378], [479, 435]]

In [46]:
npdepth = np.asarray(depth)
point_mask = np.zeros(npdepth.shape)

for p in points:
    point_mask[p[1], p[0]] = npdepth[p[1], p[0]]

markers_o3d = open3d.geometry.Image(point_mask.astype(np.float32))
# intr = Intrinsics(width=640, height=480)
intr = Intrinsics(width=1024, height=768)

m_pcd = open3d.geometry.PointCloud.create_from_depth_image(markers_o3d, intrinsic=intr.get_intrinsics())
m_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
m_pcd.paint_uniform_color([1.0, 0, 0])

points3d = np.asarray(m_pcd.points).copy()

u_1 = points3d[1] - points3d[2]
u_2 = points3d[0] - points3d[2]

n = np.cross(u_1, u_2)
R = Rotation.align_vectors([0, 1, 0], n)[0].as_matrix()

In [47]:
m_pcd.translate(-points3d[2])
m_pcd.rotate(R, center=(0, 0, 0))

PointCloud with 3 points.

In [48]:
def apply_bounding_box(pcd, roi_points):
    p1 = roi_points[np.argmin(roi_points[:, 0])]
    p2 = roi_points[np.argmin(roi_points[:, 2])]
    
    # p1 = p1 - np.array([.3, 0, 0])
    # p2 = p2 + np.array([.3, 1, -.7])
    
    # Test 1
    p1 = p1 + np.array([0, 0, -1.9])
    p2 = p2 + np.array([.5, 1, -2])    
    
    points_to_filter = np.asarray(pcd.points)
    
    x_bound = (points_to_filter[:, 0] > p1[0]) & (points_to_filter[:, 0] < p2[0])
    z_bound = (points_to_filter[:, 2] < p1[2]) & (points_to_filter[:, 2] > p2[2])
    y_bound = (points_to_filter[:, 1] > p1[1]) & (points_to_filter[:, 1] < p2[1])
    
    return pcd.select_by_index(np.where(x_bound & z_bound & y_bound)[0])

# pcd_filtered = apply_bounding_box(pcd_body, np.array(m_pcd.points))

Depth image interpolation

In [49]:
from skimage.restoration import inpaint

In [50]:
ref_with_body = np.asanyarray(open3d.io.read_image(paths[0])) * scale

In [51]:
npdepth = np.asarray(depth)
temp_mask = np.zeros(npdepth.shape)
mask = np.zeros(npdepth.shape)

temp_mask[npdepth == 0] = 1

mask[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]] = temp_mask[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]]
interpolated_depth = inpaint.inpaint_biharmonic(ref_with_body, mask)

In [52]:
plt.imshow(npdepth)
plt.show()

In [53]:
ref_with_body = open3d.geometry.Image(interpolated_depth.astype(np.float32))
ref_with_body_pcd = open3d.geometry.PointCloud.create_from_depth_image(depth=ref_with_body, intrinsic=intr.get_intrinsics())
ref_with_body_pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
ref_with_body_pcd.translate(-points3d[2])
ref_with_body_pcd.rotate(R, center=(0, 0, 0))

ref_with_only_body_pcd = apply_bounding_box(ref_with_body_pcd, np.array(m_pcd.points))

In [54]:
axes = open3d.geometry.TriangleMesh.create_coordinate_frame()

In [55]:
open3d.visualization.draw_geometries([ref_with_body_pcd, axes])

In [56]:
open3d.visualization.draw_geometries([ref_with_only_body_pcd, axes])

<h3>Video</h3>

In [57]:
vis = open3d.visualization.Visualizer()
vis.create_window()

geometry = open3d.geometry.PointCloud()
vis.add_geometry(geometry)
vis.add_geometry(axes)

fps=30
img=None

diff = []
for i, path in tqdm(enumerate(paths[350:820]), total=820-350):
    depth = np.asanyarray(open3d.io.read_image(path)) * scale
    temp_mask = np.zeros(depth.shape)
    mask = np.zeros(depth.shape)
    temp_mask[depth == 0] = 1
    mask[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]] = temp_mask[roi[0][1]:roi[1][1], roi[0][0]:roi[1][0]]
    depth = inpaint.inpaint_biharmonic(depth, mask)
    
    depth = open3d.geometry.Image(depth.astype(np.float32))
    
    pcd = open3d.geometry.PointCloud.create_from_depth_image(depth=depth, intrinsic=intr.get_intrinsics())
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    pcd.translate(-points3d[2])
    pcd.rotate(R, center=(0, 0, 0))

    pcd = apply_bounding_box(pcd, np.asarray(m_pcd.points))
    dists = pcd.compute_point_cloud_distance(ref_with_only_body_pcd)
    m_dists = np.mean(np.asarray(dists))
    diff.append(m_dists)

    geometry.points = pcd.points
        
    if i == 0:
        vis.add_geometry(geometry)

    vis.update_geometry(geometry)
    vis.poll_events()
    vis.update_renderer()
    
vis.destroy_window()

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


  0%|          | 0/470 [00:00<?, ?it/s]

In [21]:
len(paths)

1938

In [58]:
from scipy.ndimage import gaussian_filter1d

fig = plt.figure()
plt.plot(np.array(diff), label='original data')
plt.plot(gaussian_filter1d(np.array(diff), sigma=2))
plt.xlabel("difference")
plt.xlabel("frame")
plt.show()

NameError: name 'vis' is not defined

In [59]:
vis.destroy_window()